In [7]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [8]:
class AgentState(TypedDict):
    Number_1: int
    Number_2: int
    Number_3: int
    Number_4: int
    Result_1: str
    Result_2: str
    operation_1: str
    operation_2: str

In [9]:
def start_node(state: AgentState) -> AgentState:
    return state

def addition_1(state: AgentState) -> AgentState:
    result = state["Number_1"] + state["Number_2"]
    state["Result_1"] = str(result)
    return state

def subtraction_1(state: AgentState) -> AgentState:
    result = state["Number_1"] - state["Number_2"]
    state["Result_1"] = str(result)
    return state

def addition_2(state: AgentState) -> AgentState:
    result = int(state["Result_1"]) + state["Number_3"] + state["Number_4"]
    state["Result_2"] = str(result)
    return state

def subtraction_2(state: AgentState) -> AgentState:
    result = int(state["Result_1"]) - state["Number_3"] - state["Number_4"]
    state["Result_2"] = str(result)
    return state

def final_node(state: AgentState) -> AgentState:
    print("Final result:", state["Result_2"])
    return state

In [10]:
builder = StateGraph(AgentState)

builder.add_node("start", start_node)
builder.add_node("add_1", addition_1)
builder.add_node("sub_1", subtraction_1)
builder.add_node("add_2", addition_2)
builder.add_node("sub_2", subtraction_2)
builder.add_node("final", final_node)

builder.set_entry_point("start")

# First conditional edge
builder.add_conditional_edges(
    "start",
    lambda state: "add_1" if state["operation_1"] == "+" else "sub_1",
    {"add_1": "add_1", "sub_1": "sub_1"}
)

# Second conditional edge after Result_1
builder.add_conditional_edges(
    "add_1",
    lambda state: "add_2" if state["operation_2"] == "+" else "sub_2",
    {"add_2": "add_2", "sub_2": "sub_2"}
)

builder.add_conditional_edges(
    "sub_1",
    lambda state: "add_2" if state["operation_2"] == "+" else "sub_2",
    {"add_2": "add_2", "sub_2": "sub_2"}
)

# Final edge
builder.add_edge("add_2", "final")
builder.add_edge("sub_2", "final")

graph = builder.compile()

In [11]:
graph.invoke({
    "Number_1": 10,
    "Number_2": 5,
    "Number_3": 2,
    "Number_4": 1,
    "operation_1": "+",
    "operation_2": "-"
})

Final result: 12


{'Number_1': 10,
 'Number_2': 5,
 'Number_3': 2,
 'Number_4': 1,
 'Result_1': '15',
 'Result_2': '12',
 'operation_1': '+',
 'operation_2': '-'}